In [ ]:
gpu_info = !nvidia-smi
gpu_info = gpu_info[:10]
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Sep 30 19:42:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import torch
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim

from tqdm.notebook import tqdm_notebook as tq

from torch.autograd import Variable
from torchvision.transforms import transforms
from torchvision.utils import save_image
from torch.optim.lr_scheduler import StepLR

In [ ]:
base_path = "/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/"
sys.path.append(base_path)
torch.manual_seed(123)

In [ ]:
from src.data_utils import LoadDataset, SavePath
from src.model import Encoder, Decoder, Discriminator
from src.config import TrainConfig

In [ ]:
args = TrainConfig( base_path,              # project directory path
                    n_epochs = 100,         # number of epochs to train (default: 100)
                    batch_size = 128,       # input batch size for training (default: 128)
                    lr = 0.0002,            # learning rate (default: 0.0001)
                    dim_h = 128,            # hidden dimension (default: 128)')
                    n_z = 8,                # hidden dimension of z (default: 8)
                    LAMBDA = 10,            # regularization coef term (default: 10)
                    sigma = 1,              # variance of hidden dimension (default: 1)
                    n_channel = 1,          # input channels (default: 1)
                    img_size = 28 )         # image size

In [ ]:
# sp = SavePath(args, checkpoint_path=None)

/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/outs/Fri-Sep-18-19-47-27-2020/


In [ ]:
# cdl = CustomDataLoader(args)
# train_loader = cdl.get_data_loader(train=True)
# test_loader = cdl.get_data_loader(train=False)

In [ ]:
# criterion = nn.MSELoss()

In [ ]:
# ae = AutoEncoderGAN(args, criterion, torch.cuda.is_available(), 
                    # train_loader, test_loader, optimizer="Adam")

In [ ]:
# ae.train(out_frequency=1, save_model_frequency=25, save_paths=sp)

In [ ]:
sp = SavePath(args, checkpoint_path="/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/outs/Wed-Sep-30-18-59-15-2020/")
# sp = SavePath(args)

/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/outs/Wed-Sep-30-18-59-15-2020/


In [ ]:
def unfreeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def freeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [ ]:
def save_models(model_path, epoch_no, encoder, decoder, discriminator):
    print("Saving models")
    if encoder:
        torch.save(encoder.state_dict(), model_path + "/encoder_" + "%d.pth" % epoch_no)
    if decoder:
        torch.save(decoder.state_dict(), model_path + "/decoder_" + "%d.pth" % epoch_no)
    if discriminator:
        torch.save(discriminator.state_dict(), model_path + "/discriminator_" + "%d.pth" % epoch_no)

def save_lists(list_path, epoch_no, reconstr_loss):
    print("Saving list")
    if reconstr_loss:
        np.savetxt(list_path + '/reconstr_loss_' + '%d.txt'% epoch_no, reconstr_loss)

In [ ]:
cdl = LoadDataset(args)
train_loader = cdl.get_data_loader(train=True)
test_loader = cdl.get_data_loader(train=False)

In [ ]:
encoder, decoder, discriminator = Encoder(args), Decoder(args), Discriminator(args)
criterion = nn.MSELoss()

encoder.train()
decoder.train()
discriminator.train()

# Optimizers
enc_optim = optim.Adam(encoder.parameters(), lr = args.lr)
dec_optim = optim.Adam(decoder.parameters(), lr = args.lr)
dis_optim = optim.Adam(discriminator.parameters(), lr = 0.5 * args.lr)

enc_scheduler = StepLR(enc_optim, step_size=30, gamma=0.5)
dec_scheduler = StepLR(dec_optim, step_size=30, gamma=0.5)
dis_scheduler = StepLR(dis_optim, step_size=30, gamma=0.5)

In [ ]:
if torch.cuda.is_available():
    encoder, decoder, discriminator = encoder.cuda(), decoder.cuda(), discriminator.cuda()

In [ ]:
one = torch.tensor(1)
mone = one * -1
if torch.cuda.is_available():
    one, mone = one.cuda(), mone.cuda()

In [29]:
checkpoint = 200

if checkpoint:
    _, list_path, model_path = sp.get_save_paths()

    encoder.load_state_dict(torch.load(
                    model_path + "/encoder_{}.pth".format(checkpoint)))
    decoder.load_state_dict(torch.load(
                    model_path + "/decoder_{}.pth".format(checkpoint)))
    discriminator.load_state_dict(torch.load(
            model_path + "/discriminator_{}.pth".format(checkpoint)))
    reconstr_loss_epoch = np.loadtxt(
        list_path + '/reconstr_loss_{}.txt'.format(checkpoint)).tolist()

if not checkpoint:
    reconstr_loss_epoch = []
reconstr_loss = []
disc_loss = []
d_real_val = []

image_path, list_path, model_path = sp.get_save_paths()

In [30]:
for epoch in range(checkpoint, checkpoint+args.n_epochs):
    for step, (images, _) in tq(enumerate(train_loader)):
        
        reconstr_loss.clear() 
        # discriminator_loss.clear()
        
        if torch.cuda.is_available():
            images = images.cuda()

        encoder.zero_grad()
        decoder.zero_grad()
        discriminator.zero_grad()

        # ======== Train Discriminator ======== #

        freeze_params(decoder)
        freeze_params(encoder)
        unfreeze_params(discriminator)

        z_fake = torch.randn(images.size()[0], args.n_z) * args.sigma

        if torch.cuda.is_available():
            z_fake = z_fake.cuda()

        d_fake = discriminator(z_fake)

        z_real = encoder(images)
        d_real = discriminator(z_real)

        # df = round(, 3)
        # dr = round(torch.true_divide((d_real > 0.5).sum(), d_real.shape[0]).item(), 3)

        # negate for gradient ascent
        d_loss = -(torch.log(d_fake).mean() + torch.log(1 - d_real).mean())

        disc_loss.append([d_fake.mean().item(), d_real.mean().item(), d_loss.item()])

        d_loss.backward()

        dis_optim.step()

        if (step + 1) % 10 == 0:

            # ======== Train Generator ======== #

            unfreeze_params(decoder)
            unfreeze_params(encoder)
            freeze_params(discriminator)

            batch_size = images.size()[0]

            z_real = encoder(images)
            x_recon = decoder(z_real)
            d_real = discriminator(encoder(Variable(images.data)))

            d_real_val.append((torch.log(d_real)).mean().item())
            # recon_loss = criterion(x_recon, images) - (args.LAMBDA * (torch.log(d_real)).mean())

            recon_loss = criterion(x_recon, images)
            d_loss = args.LAMBDA * (torch.log(d_real)).mean()

            recon_loss.backward(one)
            d_loss.backward(mone)

            enc_optim.step()
            dec_optim.step()

            reconstr_loss.append(recon_loss.data.item())

    if (epoch + 1) % 1 == 0:
        # print("Epoch: [%d/%d], Step: [%d/%d], Reconstruction Loss: %.4f" %
        #         (epoch + 1, args.n_epochs, step + 1, len(train_loader), recon_loss.data.item()))
        temp = np.mean(disc_loss, axis=0)
        print("Epoch: [%d/%d], Recon Loss: [%.4f], Reg loss: [%.4f], D_fake: [%.4f], D_real: [%.4f], D_loss: [%.4f]" %
                (epoch + 1, args.n_epochs, 
                 np.mean(reconstr_loss), np.mean(d_real_val), temp[0], temp[1], temp[2]))
        disc_loss.clear()
        d_real_val.clear()

    if (epoch + 1) % 1 == 0:

        reconstr_loss_epoch.append(np.mean(reconstr_loss))
        reconstr_loss.clear()

        batch_size = 128
        test_iter = iter(test_loader)
        test_data = next(test_iter)

        z_real = encoder(Variable(test_data[0]).cuda())
        reconst = decoder(z_real).cpu().view(batch_size, 1, 28, 28)
        sample = decoder(torch.randn_like(z_real)).cpu().view(batch_size, 1, 28, 28)

        image = torch.cat((test_data[0].view(batch_size, 1, 28, 28), 
                                reconst.data), axis=3)
        
        save_image(image, image_path +
                                '/inputs_reconstr_{}.png'.format(epoch+1))
        
        save_image(sample, image_path + 
                                '/sample_{}.png'.format(epoch+1))

        # save_image(test_data[0].view(batch_size, 1, 28, 28), image_path + '/wae_gan_input.png')
        # save_image(reconst.data, image_path + '/wae_gan_images_%d.png' % (epoch + 1))
    
    if (epoch + 1) % 25 == 0:
        save_models(model_path, epoch+1, encoder, decoder, discriminator)
        save_lists(list_path, epoch+1, reconstr_loss_epoch)


Epoch: [201/100], Recon Loss: [nan], Reg loss: [-0.7097], D_fake: [0.5062], D_real: [0.4949], D_loss: [1.3749]


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Epoch: [202/100], Recon Loss: [nan], Reg loss: [-0.7193], D_fake: [0.5050], D_real: [0.4917], D_loss: [1.3733]



Epoch: [203/100], Recon Loss: [nan], Reg loss: [-0.7117], D_fake: [0.5058], D_real: [0.4931], D_loss: [1.3740]



Epoch: [204/100], Recon Loss: [nan], Reg loss: [-0.7125], D_fake: [0.5040], D_real: [0.4944], D_loss: [1.3777]


...


Epoch: [296/100], Recon Loss: [nan], Reg loss: [-0.7101], D_fake: [0.5059], D_real: [0.4942], D_loss: [1.3749]



Epoch: [297/100], Recon Loss: [nan], Reg loss: [-0.7104], D_fake: [0.5060], D_real: [0.4956], D_loss: [1.3758]



Epoch: [298/100], Recon Loss: [nan], Reg loss: [-0.7085], D_fake: [0.5047], D_real: [0.4944], D_loss: [1.3758]



Epoch: [299/100], Recon Loss: [nan], Reg loss: [-0.7088], D_fake: [0.5031], D_real: [0.4951], D_loss: [1.3785]



Epoch: [300/100], Recon Loss: [nan], Reg loss: [-0.7043], D_fake: [0.5043], D_real: [0.4969], D_loss: [1.3791]
Saving models
Saving list


In [ ]:
reconstr_loss

[]

In [ ]:
z_real = encoder(Variable(test_data[0]).cuda())        
reconst = decoder(z_real).cpu().view(batch_size, 1, 28, 28)
sample = decoder(torch.randn_like(z_real)).cpu().view(batch_size, 1, 28, 28)

RuntimeError: ignored

In [ ]:
image = torch.cat((test_data[0].view(batch_size, 1, 28, 28), 
                                reconst.data), axis=3)

In [ ]:
image.shape

torch.Size([128, 1, 28, 56])

In [ ]:
np.mean(disc_loss, axis=0)

In [ ]:
disc_loss[0]

Output hidden; open in https://colab.research.google.com to view.